# Generate filter calibration curves
This script runs the monochromator through a series of wavelength steps, taking flats at each step to provide filter transmission curves for each filter.

In [ ]:
import sys
import asyncio
import time
import os
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.time import Time
from astropy.table import Table

from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.latiss import LATISS

In [ ]:
# change this to a local directory
data_dir = '/home/craiglagegit/DATA'

## Now connect to LATISS and the ATCalSys CSCs

In [ ]:
domain = salobj.Domain()
latiss = LATISS(domain)
atmonochromator = salobj.Remote(domain, 'ATMonochromator')
FiberSpectrograph = salobj.Remote(name="FiberSpectrograph", domain=domain, index=3)
electrometer = salobj.Remote(name="Electrometer", domain=domain, index=201) 
WhiteLightSource = salobj.Remote(name='ATWhiteLight', domain=domain)

## Bring in needed subroutines

In [ ]:
# functions to run the electrometer measurements
def get_file_transfer(lfa):
    filename = os.path.split(lfa.url)[1]
    save_dir = os.path.join(data_dir, filename.split(':')[0])
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_file = os.path.join(save_dir, filename)
    os.system(f"curl {lfa.url} --output {save_file}")
    return save_file

async def elec_meas(exp_time):
    electrometer.evt_largeFileObjectAvailable.flush()
    tmp1=await electrometer.cmd_startScan.set_start(timeout=10)
    await asyncio.sleep(exp_time)
    tmp2=await electrometer.cmd_stopScan.set_start(timeout=10)
    lfa = await electrometer.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
    filename = get_file_transfer(lfa)
    filename = filename.replace(":", "_")
    elec_filen = os.path.join(data_dir, filename)
    os.system(f"curl {lfa.url} --output {elec_filen}")
    hdu = fits.open(f'{elec_filen}')
    data = hdu[1].data
    return filename, lfa, data

async def fiber_meas(exp_time):
    FiberSpectrograph.evt_largeFileObjectAvailable.flush()
    tmp1 = await FiberSpectrograph.cmd_expose.set_start(duration=exp_time, numExposures=1)
    lfa = await FiberSpectrograph.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
    filename = get_file_transfer(lfa)
    filename = filename.replace(":", "_")
    spec_filen = os.path.join(data_dir, filename)
    os.system(f"curl {lfa.url} --output {spec_filen}")
    hdu = fits.open(f'{spec_filen}')
    wavelength = hdu[1].data['wavelength'].flatten()
    spectrum = hdu[0].data
    return filename, lfa, wavelength, spectrum


## Set up electrometer

In [ ]:
await electrometer.cmd_performZeroCalib.set_start(timeout=10)
await electrometer.cmd_setDigitalFilter.set_start(activateFilter=False, activateAvgFilter=False, activateMedFilter=False, timeout=10)  
await electrometer.cmd_setIntegrationTime.set_start(intTime=0.1) 
await electrometer.cmd_setRange.set_start(setRange=-1)

## Take 20 biases for system warm-up

In [ ]:
# Take 20 biases for system warm-up
for i in range(20):
    await latiss.take_bias(1)
    await asyncio.sleep(2.0)


## Now run SDSSr filter

### 202312190024 is the first exposure

In [ ]:
latiss_grating= 'empty_1'
wave_step = 5 # Wavelength step in nm
latiss_exp_time = 60.0
num_latiss_exps = 1

# Dictionary to hold the results
results = {}
for latiss_filter in ['SDSSr_65mm']:#['SDSSg_65mm', 'SDSSr_65mm', 'SDSSi_65mm']:
    if latiss_filter =='SDSSg_65mm':
        min_wave = 350; max_wave = 600; grating = 0; slit_1 = 5.0; slit_2 = 5.0
    elif latiss_filter == 'SDSSr_65mm':
        min_wave = 500; max_wave = 750; grating = 1; slit_1 = 5.0; slit_2 = 5.0
    elif latiss_filter == 'SDSSi_65mm':
        min_wave = 640; max_wave = 880; grating = 1; slit_1 = 5.0; slit_2 = 5.0
    else:
        raise "Filter not supported"
    await atmonochromator.cmd_updateMonochromatorSetup.set_start(wavelength=min_wave, gratingType=grating, \
                                                             fontEntranceSlitWidth=slit_1, fontExitSlitWidth=slit_2, timeout=180)        
    wave = min_wave
    # This loop steps through the wavelengths for each filter
    while wave < max_wave + wave_step / 2.0:
        await atmonochromator.cmd_updateMonochromatorSetup.set_start(wavelength=wave, gratingType=grating, \
                                                             fontEntranceSlitWidth=slit_1, fontExitSlitWidth=slit_2, timeout=180)        
        key = f"{latiss_filter}_{wave:.0f}"
        print(key)
        filename, electrometer_lfa, electrometer_data = await elec_meas(1.0)
        for (t,current) in electrometer_data:
            electrometer_current = current
        exp_time = 0.5
        filename, fiber_lfa, wavelengths, spectrum = await fiber_meas(exp_time)
        spec_max = np.max(spectrum)
        # The loop below prevents the spectrum from saturating
        while spec_max > 15900:
            exp_time /= 2.0
            filename, fiber_lfa, wavelengths, spectrum = await fiber_meas(exp_time)
            spec_max = np.max(spectrum)
        # Take one flat with the filter and one without
        await latiss.take_flats(latiss_exp_time, num_latiss_exps, filter=latiss_filter, grating=latiss_grating, reason=f'flat_{wave:.1f}')
        await latiss.take_flats(latiss_exp_time, num_latiss_exps, filter='empty_1', grating=latiss_grating, reason=f'flat_{wave:.1f}')
        
        print(wave, electrometer_current, exp_time, spec_max)
        results[key] = {}
        results[key]['electrometer_lfa_url']=electrometer_lfa.url
        results[key]['fiber_lfa_url']=fiber_lfa.url
        results[key]['electrometer_current']=electrometer_current
        results[key]['fiber_data']=[wavelengths, spectrum, exp_time]
        print(f"Done with {latiss_filter}, {wave}")
        wave += wave_step

In [ ]:
# Pickle the data
filename = "/scratch/cslage/filter_cals/filter_cal_19Dec23.pkl"
file = open(filename, "wb")
pkl.dump(results, file)
file.close()

In [ ]:
# Turn off the lamp
tmp = await WhiteLightSource.cmd_turnLampOff.set_start()

## Below shows how to unpickle the spectrum and electrometer data and plot a spectrum

In [ ]:
# Unpickle the data
filename = "/scratch/cslage/filter_cals/filter_cal_19Dec23.pkl"
file = open(filename, "rb")
results = pkl.load(file)
file.close()

In [ ]:
[wavelengths, spectrum, exp_time] = results['SDSSr_65mm_610']['fiber_data']

In [ ]:
%matplotlib inline
plt.plot(wavelengths, spectrum)
plt.xlim(560, 660)

In [ ]:
current = results['SDSSr_65mm_610']['electrometer_current']

In [ ]:
current